### Tugas Praktikum

In [50]:
import tensorflow as tf
import numpy as np
import os
import time

#### Praktikum 2

In [51]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [52]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [53]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [54]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [55]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [56]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [57]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [58]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [59]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [60]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [61]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [62]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [63]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [64]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [65]:
seq_length = 100

In [66]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [67]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [68]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [69]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [70]:
dataset = sequences.map(split_input_target)

In [71]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [72]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [73]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [74]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [75]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [76]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [77]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [78]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [79]:
sampled_indices

array([26, 23, 51, 44, 54, 44, 51,  0, 38,  9, 54, 11, 30, 15, 14,  7, 63,
       13,  8, 50, 49,  3, 11, 31, 43, 52, 45,  3, 12, 53, 42,  2, 53, 41,
       38,  1, 51,  5,  4, 20, 30, 64, 60,  3, 33, 23, 43, 55, 32, 34, 14,
       56, 31, 60, 53, 15,  1, 28, 14, 36, 34, 13, 25, 56, 56, 50, 36, 16,
       42, 31, 49, 23, 51, 46, 56, 57, 10, 51, 53, 50, 33,  8,  7, 10, 46,
       18,  8, 40, 23, 63, 48, 17, 38,  1, 64, 60, 39,  3, 37,  1])

In [80]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'nap.\nBut did I never speak of all that time?\n\nFirst Servant:\nO, yes, my lord, but very idle words:\nF'

Next Char Predictions:
 b'MJleoel[UNK]Y.o:QBA,x?-kj!:Rdmf!;nc nbY\nl&$GQyu!TJdpSUAqRunB\nOAWU?LqqkWCcRjJlgqr3lnkT-,3gE-aJxiDY\nyuZ!X\n'


In [81]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [82]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1905575, shape=(), dtype=float32)


In [83]:
tf.exp(example_batch_mean_loss).numpy()

66.05961

In [84]:
model.compile(optimizer='adam', loss=loss)

In [85]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [49]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 60ms/step - loss: 2.6791
Epoch 2/20
172/172 [==============================] - 15s 62ms/step - loss: 1.9517
Epoch 3/20
172/172 [==============================] - 14s 62ms/step - loss: 1.6752
Epoch 4/20
172/172 [==============================] - 13s 61ms/step - loss: 1.5210
Epoch 5/20
172/172 [==============================] - 12s 58ms/step - loss: 1.4285
Epoch 6/20
172/172 [==============================] - 11s 58ms/step - loss: 1.3641
Epoch 7/20
172/172 [==============================] - 12s 59ms/step - loss: 1.3127
Epoch 8/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2691
Epoch 9/20
172/172 [==============================] - 14s 61ms/step - loss: 1.2287
Epoch 10/20
172/172 [==============================] - 12s 63ms/step - loss: 1.1882
Epoch 11/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1468
Epoch 12/20
172/172 [==============================] - 13s 60ms/step - loss: 1.1042
E

#### Praktikum Tugas

In [88]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)

      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [89]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [90]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [91]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 60ms/step - loss: 2.7021


In [92]:
EPOCHS = 20

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)


    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1773
Epoch 1 Batch 50 Loss 2.0755
Epoch 1 Batch 100 Loss 1.9566
Epoch 1 Batch 150 Loss 1.8710

Epoch 1 Loss: 1.9753
Time taken for 1 epoch 12.76 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8026
Epoch 2 Batch 50 Loss 1.7646
Epoch 2 Batch 100 Loss 1.6850
Epoch 2 Batch 150 Loss 1.6484

Epoch 2 Loss: 1.6997
Time taken for 1 epoch 11.35 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5591
Epoch 3 Batch 50 Loss 1.5372
Epoch 3 Batch 100 Loss 1.5203
Epoch 3 Batch 150 Loss 1.4999

Epoch 3 Loss: 1.5412
Time taken for 1 epoch 11.19 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4586
Epoch 4 Batch 50 Loss 1.4690
Epoch 4 Batch 100 Loss 1.4225
Epoch 4 Batch 150 Loss 1.4232

Epoch 4 Loss: 1.4431
Time taken for 1 epoch 11.05 sec
_____________________________________________________________________

- Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

> Praktikum 2 memiliki estimasi waktu yang lebih cepat untuk menjalankan proses setiap epoch daripada Praktikum Tugas. Ini karena, meskipun estimasi waktu Praktikum Tugas tetap sama, yaitu 11 hingga 14 detik, estimasi Praktikum Tugas melonjak menjadi 20.59 detik pada epoch 5.

> Namun, nilai kehilangan yang dihasilkan pada Praktikum Tugas lebih rendah daripada pada Praktikum 2. Nilai kehilangan pada Epoch 1 Praktikum 2 adalah 2.6791 dan Epoch 20 menghasilkan nilai kehilangan 0.7131, sedangkan pada Epoch Praktikum Tugas dimulai dengan nilai kehilangan 2.0049 pada Epoch 1 dan berakhir dengan nilai kehilangan 0.7067 pada Epoch 20.